In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

df = pd.read_csv('/content/최종_병합결과.csv', encoding='EUC-KR')

In [ ]:
# 날짜 컬럼을 datetime으로 변환
df["모기지수 발생일"] = pd.to_datetime(df["모기지수 발생일"])

# X: 입력 변수, y: 타겟 변수 (모기지수 3가지)
feature_cols = ['평균기온(℃)', '최고기온(℃)', '최저기온(℃)', '일교차',
                '평균습도(%rh)', '최저습도(%rh)', '평균풍속(m/s)', '강수량(mm)']
target_cols = ['모기지수(수변부)', '모기지수(주거지)', '모기지수(공원)']

X = df[feature_cols]
y = df[target_cols]

# 데이터 분할 (학습:검증 = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 회귀 모델 (Random Forest 기반 MultiOutput 회귀)
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train_scaled, y_train)

# 예측
y_pred = model.predict(X_test_scaled)

# 성능 평가
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

mae_dict = dict(zip(target_cols, mae))
rmse_dict = dict(zip(target_cols, rmse))

mae_dict, rmse_dict

({'모기지수(수변부)': np.float64(7.21566025641026),
  '모기지수(주거지)': np.float64(10.751474358974361),
  '모기지수(공원)': np.float64(9.048839743589744)},
 {'모기지수(수변부)': np.float64(13.64282915312224),
  '모기지수(주거지)': np.float64(14.812059879861932),
  '모기지수(공원)': np.float64(11.744106294113482)})

In [ ]:
def predict_mosquito_index(temp_avg, temp_max, temp_min, temp_range,
                           humidity_avg, humidity_min, wind_speed_avg, rainfall,
                           scaler_obj, trained_model, feature_names):
    """
    주어진 기상 조건을 사용하여 모기 지수를 예측합니다.

    Args:
        temp_avg (float): 평균 기온 (℃)
        temp_max (float): 최고 기온 (℃)
        temp_min (float): 최저 기온 (℃)
        temp_range (float): 일교차
        humidity_avg (float): 평균 습도 (%rh)
        humidity_min (float): 최저 습도 (%rh)
        wind_speed_avg (float): 평균 풍속 (m/s)
        rainfall (float): 강수량 (mm)
        scaler_obj (StandardScaler): 학습된 StandardScaler 객체.
        trained_model (MultiOutputRegressor): 학습된 MultiOutputRegressor 모델.
        feature_names (list): 모델 학습에 사용된 특성 컬럼 이름 리스트.

    Returns:
        pd.Series: 예측된 모기 지수 (수변부, 주거지, 공원)
    """
    # 입력 데이터를 DataFrame으로 변환
    input_data = pd.DataFrame([[temp_avg, temp_max, temp_min, temp_range,
                                  humidity_avg, humidity_min, wind_speed_avg, rainfall]],
                              columns=feature_names)

    # 학습 시 사용된 스케일러로 입력 데이터 스케일링
    input_scaled = scaler_obj.transform(input_data)

    # 모델로 예측 수행
    prediction = trained_model.predict(input_scaled)

    # 예측 결과를 읽기 쉬운 형태로 반환 (Series)
    return pd.Series(prediction[0], index=target_cols)


print("\n[2025년 5월 23일 (가상 데이터) 예측]")
next_day_weather1 = {
    '평균기온(℃)': 19.4,
    '최고기온(℃)': 22.2,
    '최저기온(℃)': 15.4,
    '일교차': 6.8,
    '평균습도(%rh)': 53.8,
    '최저습도(%rh)': 36.0,
    '평균풍속(m/s)': 2.8,
    '강수량(mm)': 1.2
}

predicted_index_next_day1 = predict_mosquito_index(
    temp_avg=next_day_weather1['평균기온(℃)'],
    temp_max=next_day_weather1['최고기온(℃)'],
    temp_min=next_day_weather1['최저기온(℃)'],
    temp_range=next_day_weather1['일교차'],
    humidity_avg=next_day_weather1['평균습도(%rh)'],
    humidity_min=next_day_weather1['최저습도(%rh)'],
    wind_speed_avg=next_day_weather1['평균풍속(m/s)'],
    rainfall=next_day_weather1['강수량(mm)'],
    scaler_obj=scaler,
    trained_model=model,
    feature_names=feature_cols
)
print(predicted_index_next_day1)
print("\n[2025년 5월 24일 (가상 데이터) 예측]")
next_day_weather2 = {
    '평균기온(℃)': 15.6,
    '최고기온(℃)': 19.3,
    '최저기온(℃)': 13.0,
    '일교차': 6.3,
    '평균습도(%rh)': 69.1,
    '최저습도(%rh)': 53.0,
    '평균풍속(m/s)': 3.1,
    '강수량(mm)': 4.3
}

predicted_index_next_day2 = predict_mosquito_index(
    temp_avg=next_day_weather2['평균기온(℃)'],
    temp_max=next_day_weather2['최고기온(℃)'],
    temp_min=next_day_weather2['최저기온(℃)'],
    temp_range=next_day_weather2['일교차'],
    humidity_avg=next_day_weather2['평균습도(%rh)'],
    humidity_min=next_day_weather2['최저습도(%rh)'],
    wind_speed_avg=next_day_weather2['평균풍속(m/s)'],
    rainfall=next_day_weather2['강수량(mm)'],
    scaler_obj=scaler,
    trained_model=model,
    feature_names=feature_cols
)
print(predicted_index_next_day2)
print("\n[2025년 5월 25일 (가상 데이터) 예측]")
next_day_weather3 = {
    '평균기온(℃)': 17.4,
    '최고기온(℃)': 22.6,
    '최저기온(℃)': 11.7,
    '일교차': 10.9,
    '평균습도(%rh)': 61.0,
    '최저습도(%rh)': 44.0,
    '평균풍속(m/s)': 2.1,
    '강수량(mm)': 0
}

predicted_index_next_day3 = predict_mosquito_index(
    temp_avg=next_day_weather3['평균기온(℃)'],
    temp_max=next_day_weather3['최고기온(℃)'],
    temp_min=next_day_weather3['최저기온(℃)'],
    temp_range=next_day_weather3['일교차'],
    humidity_avg=next_day_weather3['평균습도(%rh)'],
    humidity_min=next_day_weather3['최저습도(%rh)'],
    wind_speed_avg=next_day_weather3['평균풍속(m/s)'],
    rainfall=next_day_weather3['강수량(mm)'],
    scaler_obj=scaler,
    trained_model=model,
    feature_names=feature_cols
)
print(predicted_index_next_day3)


[2025년 5월 23일 (가상 데이터) 예측]
모기지수(수변부)    87.066
모기지수(주거지)    44.253
모기지수(공원)     36.404
dtype: float64

[2025년 5월 24일 (가상 데이터) 예측]
모기지수(수변부)    73.426
모기지수(주거지)    37.297
모기지수(공원)     27.888
dtype: float64

[2025년 5월 25일 (가상 데이터) 예측]
모기지수(수변부)    61.171
모기지수(주거지)    20.698
모기지수(공원)     26.069
dtype: float64
